# చాట్ చరిత్ర తగ్గింపు మరియు ఏజెంట్ స్క్రాచ్‌ప్యాడ్‌ను సెమాంటిక్ కర్నెల్‌లో ఉపయోగించడం

ఈ నోట్‌బుక్ సెమాంటిక్ కర్నెల్ యొక్క చాట్ చరిత్ర తగ్గింపు ఫీచర్‌ను ఏజెంట్ స్క్రాచ్‌ప్యాడ్‌తో కలిపి సంభాషణలలో కంటెక్స్ట్‌ను నిర్వహించడానికి ఎలా ఉపయోగించాలో చూపిస్తుంది. ఇది టోకెన్ పరిమితులను మించకుండా దీర్ఘ సంభాషణలను నిర్వహించగల సమర్థవంతమైన AI ఏజెంట్లను నిర్మించడానికి అవసరం.

## మీరు నేర్చుకునే విషయాలు:
1. **చాట్ చరిత్ర తగ్గింపు**: టోకెన్ వినియోగాన్ని నిర్వహించడానికి సంభాషణ చరిత్రను స్వయంచాలకంగా సారాంశం చేయడం
2. **ఏజెంట్ స్క్రాచ్‌ప్యాడ్**: వినియోగదారుల అభిరుచులు మరియు పూర్తి చేసిన పనులను ట్రాక్ చేయడానికి స్థిరమైన మెమరీ వ్యవస్థ
3. **టోకెన్ వినియోగ ట్రాకింగ్**: చరిత్ర తగ్గింపు ఉన్నప్పుడు మరియు లేకపోయినప్పుడు టోకెన్ వినియోగం ఎలా మారుతుందో పర్యవేక్షించడం

## అవసరమైనవి:
- వాతావరణ వేరియబుల్స్ కాన్ఫిగర్ చేయబడిన Azure OpenAI సెటప్
- గత పాఠాల నుండి ప్రాథమిక ఏజెంట్ కాన్సెప్ట్‌లను అర్థం చేసుకోవడం


## అవసరమైన ప్యాకేజీలను దిగుమతి చేసుకోండి


In [ ]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## ఏజెంట్ స్క్రాచ్‌ప్యాడ్‌ను అర్థం చేసుకోవడం

### ఏజెంట్ స్క్రాచ్‌ప్యాడ్ అంటే ఏమిటి?

**ఏజెంట్ స్క్రాచ్‌ప్యాడ్** అనేది ఏజెంట్లు ఉపయోగించే ఒక స్థిరమైన మెమరీ వ్యవస్థ, ఇది:
- **పూర్తయిన పనులను ట్రాక్ చేయడం**: వినియోగదారుని కోసం ఏమి చేయబడిందో నమోదు చేయడం
- **వినియోగదారు అభిరుచులను నిల్వ చేయడం**: ఇష్టాలు, అభిరుచులు, మరియు అవసరాలను గుర్తుంచుకోవడం
- **సందర్భాన్ని నిర్వహించడం**: సంభాషణల మధ్య ముఖ్యమైన సమాచారాన్ని అందుబాటులో ఉంచడం
- **పునరావృతాన్ని తగ్గించడం**: అదే ప్రశ్నలను మళ్లీ మళ్లీ అడగకుండా ఉండడం

### ఇది ఎలా పనిచేస్తుంది:
1. **రాయడం**: ఏజెంట్ కొత్త సమాచారం నేర్చుకున్న తర్వాత స్క్రాచ్‌ప్యాడ్‌ను అప్‌డేట్ చేస్తుంది
2. **చదవడం**: నిర్ణయాలు తీసుకునేటప్పుడు ఏజెంట్ స్క్రాచ్‌ప్యాడ్‌ను పరిశీలిస్తుంది
3. **స్థిరత్వం**: చాట్ చరిత్ర తగ్గించినప్పటికీ సమాచారం నిల్వ ఉంటుంది

దీనిని సంభాషణ చరిత్రకు తోడుగా ఉండే ఏజెంట్ యొక్క వ్యక్తిగత నోట్బుక్‌గా భావించండి.


## పర్యావరణ కాన్ఫిగరేషన్


In [ ]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

## ఏజెంట్ స్క్రాచ్‌ప్యాడ్ ప్లగిన్ సృష్టించండి

ఈ ప్లగిన్ ఏజెంట్‌కు ఒక స్థిరమైన స్క్రాచ్‌ప్యాడ్ ఫైల్‌ను చదవడం మరియు రాయడం అనుమతిస్తుంది.


In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

## చాట్ చరిత్ర తగ్గింపును ప్రారంభించండి

ChatHistorySummarizationReducer చర్చ చరిత్ర ఒక పరిమితిని మించినప్పుడు స్వయంచాలకంగా చరిత్రను సారాంశం చేస్తుంది.


In [ ]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 15    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

## సెలవుల ప్రణాళిక ఏజెంట్‌ను సృష్టించండి

ఈ ఏజెంట్ వినియోగదారులు సెలవులను ప్రణాళిక చేయడంలో సహాయపడతాడు, స్క్రాచ్‌పాడ్ ద్వారా సందర్భాన్ని కొనసాగిస్తూ.


In [ ]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    CRITICAL SCRATCHPAD RULES - YOU MUST FOLLOW THESE:
    1. FIRST ACTION: When starting ANY conversation, immedi ately call read_scratchpad() to check existing preferences
    2. AFTER LEARNING PREFERENCES: When user mentions ANY preference (destinations, activities, budget, dates), 
       immediately call update_scratchpad() with category 'preferences'
    3. AFTER COMPLETING TASKS: When you finish creating an itinerary or completing any task,
       immediately call update_scratchpad() with category 'tasks'
    4. BEFORE NEW ITINERARY: Always call read_scratchpad() before creating any itinerary
    
    EXAMPLES OF WHEN TO UPDATE SCRATCHPAD:
    - User says "I love beaches" → update_scratchpad('preferences', 'Loves beach destinations')
    - User says "budget is $3000" → update_scratchpad('preferences', 'Budget: $3000 per person for a week')
    - You create an itinerary → update_scratchpad('tasks', 'Created Bali itinerary for beach vacation')
    
    PLANNING PROCESS:
    1. Read scratchpad first
    2. Ask about preferences if not found
    3. Update scratchpad with new information
    4. Create detailed itineraries
    5. Update scratchpad with completed tasks
    
    BE EXPLICIT: Always announce when you're checking or updating the scratchpad.
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with enhanced scratchpad instructions")

## ప్రదర్శన మరియు టోకెన్ ట్రాకింగ్ కోసం సహాయక ఫంక్షన్లు


In [ ]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

## సెలవుల ప్రణాళిక సంభాషణను నిర్వహించండి

ఇప్పుడు పూర్తిస్థాయి సంభాషణను నిర్వహిద్దాం, ఇది చూపిస్తుంది:
1. ప్రారంభ ప్రణాళిక అభ్యర్థన
2. అభిరుచుల సేకరణ
3. ప్రణాళిక తయారీ
4. ప్రదేశం మార్పు
5. చాట్ చరిత్ర తగ్గింపు
6. స్క్రాచ్‌ప్యాడ్ ఉపయోగం


In [ ]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0
    scratchpad_operations = 0  # Track scratchpad usage

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None
        function_calls = []  # Track function calls

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status and look for scratchpad operations
        print(f"📝 Thread has {len(thread)} messages")

        # Count scratchpad operations in this turn
        turn_scratchpad_ops = 0
        async for msg in thread.get_messages():
            if hasattr(msg, 'content') and msg.content:
                content_str = str(msg.content)
                if 'read_scratchpad' in content_str or 'update_scratchpad' in content_str:
                    turn_scratchpad_ops += 1

        if turn_scratchpad_ops > scratchpad_operations:
            print(
                f"   📝 Scratchpad operations detected: {turn_scratchpad_ops - scratchpad_operations} new operations")
            scratchpad_operations = turn_scratchpad_ops

        # Show message types for first message
        if i == 0:
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types: {message_types[:10]}..." if len(
                message_types) > 10 else f"   Message types: {message_types}")

        # Check if reduction should happen
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

            # Attempt reduction
            is_reduced = await thread.reduce()
            if is_reduced:
                print(
                    f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
                token_tracker.mark_reduction(message_count + 1)

                # Show summary if available
                async for msg in thread.get_messages():
                    if msg.metadata and msg.metadata.get("__summary__"):
                        display_message("System Summary", str(
                            msg.content), "#ff6b6b")
                        break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    print(f"\n📊 Total scratchpad operations: {scratchpad_operations}")

    return thread

# Run the conversation
thread = await run_vacation_planning()

## ఫలితాలను విశ్లేషించడం

మా సంభాషణ సమయంలో ఏమి జరిగింది అనేది విశ్లేషిద్దాం:


In [ ]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

## ముఖ్యమైన విషయాలు

### 1. చాట్ చరిత్ర తగ్గింపు
- **ఆటోమేటిక్ ట్రిగ్గరింగ్**: సందేశాల సంఖ్య పరిమితిని మించినప్పుడు తగ్గింపు జరుగుతుంది
- **టోకెన్ పొదుపు**: సారాంశం తర్వాత టోకెన్ వినియోగంలో గణనీయమైన తగ్గింపు
- **సందర్భం పరిరక్షణ**: ముఖ్యమైన సమాచారం సారాంశాల్లో నిల్వ ఉంటుంది

### 2. ఏజెంట్ స్క్రాచ్‌ప్యాడ్ ప్రయోజనాలు
- **స్థిరమైన మెమరీ**: చరిత్ర తగ్గింపు తర్వాత కూడా వినియోగదారు అభిరుచులు నిల్వ ఉంటాయి
- **పనుల ట్రాకింగ్**: ఏజెంట్ పూర్తి చేసిన పనుల రికార్డును నిర్వహిస్తుంది
- **మెరుగైన అనుభవం**: అభిరుచులను మళ్లీ చెప్పాల్సిన అవసరం లేదు

### 3. టోకెన్ వినియోగ నమూనాలు
- **లీనియర్ వృద్ధి**: ప్రతి సందేశంతో టోకెన్లు పెరుగుతాయి
- **గణనీయమైన తగ్గింపు**: తగ్గింపు టోకెన్ సంఖ్యను గణనీయంగా తగ్గిస్తుంది
- **సుస్థిరమైన సంభాషణలు**: పరిమితులలో ఎక్కువ కాలం సంభాషణలను సాధ్యమవుతుంది


## శుభ్రపరచడం

ఈ డెమో సమయంలో సృష్టించిన స్క్రాచ్‌ప్యాడ్ ఫైల్‌ను శుభ్రపరచండి:


In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# సారాంశం

అభినందనలు! మీరు అధునాతన సందర్భ నిర్వహణ సామర్థ్యాలతో కూడిన AI ఏజెంట్‌ను విజయవంతంగా అమలు చేశారు:

## మీరు నేర్చుకున్నవి:
- **చాట్ చరిత్ర తగ్గింపు**: టోకెన్ పరిమితులను నిర్వహించడానికి సంభాషణలను స్వయంచాలకంగా సారాంశం చేయడం
- **ఏజెంట్ స్క్రాచ్‌ప్యాడ్**: వినియోగదారు అభిరుచులు మరియు పూర్తయిన పనుల కోసం స్థిరమైన మెమరీ అమలు చేయడం
- **టోకెన్ నిర్వహణ**: దీర్ఘకాలిక సంభాషణల్లో టోకెన్ వినియోగాన్ని ట్రాక్ చేసి ఆప్టిమైజ్ చేయడం
- **సందర్భ పరిరక్షణ**: సంభాషణ తగ్గింపుల సమయంలో ముఖ్యమైన సమాచారాన్ని నిలుపుకోవడం

## వాస్తవ ప్రపంచ అనువర్తనాలు:
- **కస్టమర్ సర్వీస్ బాట్స్**: సెషన్ల మధ్య కస్టమర్ అభిరుచులను గుర్తుంచుకోవడం
- **వ్యక్తిగత సహాయకులు**: ప్రస్తుత ప్రాజెక్టులు మరియు వినియోగదారు అలవాట్లను ట్రాక్ చేయడం
- **శిక్షణా ఉపాధ్యాయులు**: విద్యార్థుల పురోగతి మరియు అభ్యాస అభిరుచులను నిలుపుకోవడం
- **ఆరోగ్య సంరక్షణ సహాయకులు**: టోకెన్ పరిమితులను గౌరవిస్తూ రోగి చరిత్రను నిలుపుకోవడం

## తదుపరి దశలు:
- మరింత ఆధునికమైన స్క్రాచ్‌ప్యాడ్ స్కీమాలను అమలు చేయండి
- బహుళ వినియోగదారుల పరిస్థితుల కోసం డేటాబేస్ నిల్వను జోడించండి
- డొమైన్-స్పెసిఫిక్ అవసరాల కోసం అనుకూల తగ్గింపు వ్యూహాలను సృష్టించండి
- సెమాంటిక్ మెమరీ శోధన కోసం వెక్టర్ డేటాబేస్‌లతో కలపండి
- పూర్తి సందర్భంతో రోజులు తర్వాత సంభాషణలను పునఃప్రారంభించగల ఏజెంట్లను నిర్మించండి


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్వీకరణ**:  
ఈ పత్రం AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నిస్తున్నప్పటికీ, ఆటోమేటెడ్ అనువాదాలు తప్పులు లేదా అసమగ్రతలను కలిగి ఉండవచ్చు. దాని స్వదేశ భాషలో ఉన్న అసలు పత్రాన్ని అధికారం కలిగిన మూలంగా పరిగణించాలి. కీలకమైన సమాచారం కోసం, ప్రొఫెషనల్ మానవ అనువాదాన్ని సిఫారసు చేస్తాము. ఈ అనువాదం ఉపయోగం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారులు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
